In [139]:
from datetime import datetime, timedelta
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY
from client import FtxClient

ModuleNotFoundError: No module named 'ciso8601'

In [11]:
url = 'https://ftx.com/api/markets'

#get BTCUSD spot price
base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']



In [158]:
def historical_bymonth(y, m, res="hourly"):
    """_summary_

    Args:
        y (_type_): _description_
        m (_type_): _description_
    """
    
    resolution = {
        
        "hourly":str(60*60),
        "daily":str(60*60*24)
    }
    start_date = datetime(year=y, month=m, day=1).timestamp()
    end_date = (datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(seconds=15)).timestamp()
    
    historical = requests.get(
                f'{request_url}/candles?resolution={resolution[res]}&start_time={start_date}&end_time={end_date}'
                ).json()
    
    ls = historical['result']
    # df = pd.DataFrame(historical['result'])
    
    return ls

def get_multiple_months(y,m,n_months):
    """_summary_

    Args:
        y (_type_): _description_
        m (_type_): _description_
        n_months (_type_): _description_
    """
    
    MAX_PING_PER_SEC = 12
    
    ls = []
    
    
    for _ in range(n_months):
        # if df.empty(): 
        #     df = historical_bymonth(y, m)
        # else: 
        #     df.append(historical_bymonth(y, m))
        ls.extend(historical_bymonth(y, m))
        
        if m % 12 == 0:
            y+=1
            
        m = (m % 12)+1
    

        time.sleep(1.0 / float(MAX_PING_PER_SEC))
    
    return ls

ls = get_multiple_months(2021,1,18)
df = pd.DataFrame(ls)
df

df['date'] = pd.to_datetime(
    df['time']/1000, unit='s', origin='unix'
)

#remove unnecessary columns
df.drop(['startTime','time'], axis=1, inplace=True)

df    
       
        

# df2 = historical_bymonth(2021,6,res="thirty")
# df2
# y = 2021
# m = 12
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1))
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1))
# print((datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1)).timestamp())
    

,open,high,low,close,volume,date
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00
...,...,...,...,...,...,...
13098,18869.0,20055.0,18857.0,19991.0,1.904991e+08,2022-06-30 23:00:00
13099,19998.0,20915.0,19652.0,20269.0,9.872480e+07,2022-07-01 00:00:00
13100,20270.0,20446.0,20230.0,20386.0,4.064504e+07,2022-07-01 01:00:00
13101,20387.0,20516.0,20240.0,20271.0,3.107375e+07,2022-07-01 02:00:00


In [159]:
def ema(df,length):
    """_summary_

    Args:
        df (_type_): _description_
        length (_type_): _description_
    """
    return ema


#add ohlc4 column
df['ohlc4'] = (df['open']+df['high']+df['low']+df['close'])/4

#add SMA column 
df['SMA'] = df['ohlc4'].rolling(window = 48).mean()


#add 25x liquidation prices (long/short)
sma = df['SMA']
df['25x_liq_long'] = ((sma*25)/(25+1-(0.005*25)))
df['25x_liq_short'] = ((sma*25)/(25+1+(0.005*25)))


df


,open,high,low,close,volume,date,ohlc4,SMA,25x_liq_long,25x_liq_short
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00,29240.750,NaN,NaN,NaN
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00,29228.375,NaN,NaN,NaN
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00,29131.875,NaN,NaN,NaN
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00,29102.125,NaN,NaN,NaN
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00,29189.250,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13098,18869.0,20055.0,18857.0,19991.0,1.904991e+08,2022-06-30 23:00:00,19443.000,19730.453125,19063.239734,18880.816388
13099,19998.0,20915.0,19652.0,20269.0,9.872480e+07,2022-07-01 00:00:00,20208.500,19729.557292,19062.374195,18879.959131
13100,20270.0,20446.0,20230.0,20386.0,4.064504e+07,2022-07-01 01:00:00,20333.000,19730.432292,19063.219605,18880.796451
13101,20387.0,20516.0,20240.0,20271.0,3.107375e+07,2022-07-01 02:00:00,20353.500,19731.098958,19063.863728,18881.434410


In [160]:
import pandas_datareader as pdr
import datetime as dt
import seaborn as sns



In [161]:
#create hour column
df['hour']=df.date.dt.hour
df['hour']=df['hour'].astype(float)




In [164]:
#date for volatility
data = df.set_index('date')

high_low = data['high']-data['low']
high_cp = np.abs(data['high']-data['close'].shift())
low_cp = np.abs(data['low']-data['close'].shift())

df_volatility = pd.concat([high_low, high_cp, low_cp], axis =1)

#calc true range
true_range = np.max(df_volatility, axis =1)

#calc avg true range
average_true_range = true_range.rolling(14).mean()

data['average_true_range']=average_true_range


# data.drop(['ATR'], axis=1, inplace=True)

data = data.reset_index()

data = data[['date','hour','open','high','low','close','volume','ohlc4', 'SMA', '25x_liq_long', '25x_liq_short', 'average_true_range']]
data


,date,hour,open,high,low,close,volume,ohlc4,SMA,25x_liq_long,25x_liq_short,average_true_range
0,2021-01-01 05:00:00,5.0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,29240.750,NaN,NaN,NaN,NaN
1,2021-01-01 06:00:00,6.0,29240.0,29315.0,29127.5,29231.0,7.436047e+06,29228.375,NaN,NaN,NaN,NaN
2,2021-01-01 07:00:00,7.0,29231.0,29241.5,28887.0,29168.0,8.430755e+06,29131.875,NaN,NaN,NaN,NaN
3,2021-01-01 08:00:00,8.0,29168.0,29251.0,28928.5,29061.0,7.003067e+06,29102.125,NaN,NaN,NaN,NaN
4,2021-01-01 09:00:00,9.0,29061.0,29379.5,29041.5,29275.0,5.764370e+06,29189.250,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13098,2022-06-30 23:00:00,23.0,18869.0,20055.0,18857.0,19991.0,1.904991e+08,19443.000,19730.453125,19063.239734,18880.816388,313.142857
13099,2022-07-01 00:00:00,0.0,19998.0,20915.0,19652.0,20269.0,9.872480e+07,20208.500,19729.557292,19062.374195,18879.959131,390.142857
13100,2022-07-01 01:00:00,1.0,20270.0,20446.0,20230.0,20386.0,4.064504e+07,20333.000,19730.432292,19063.219605,18880.796451,394.357143
13101,2022-07-01 02:00:00,2.0,20387.0,20516.0,20240.0,20271.0,3.107375e+07,20353.500,19731.098958,19063.863728,18881.434410,395.357143


In [165]:
#save as csv
data.to_csv("2021_2022_FTX_USDBTC_1H.csv", index=False, header=True)